Cài đặt Shadcn/UI Component:

In [ ]:
npx shadcn@latest add button input label textarea dialog table card badge

1. Cấu hình `baseQuery.js` để hỗ trợ Param (Phân trang)

In [ ]:
// src/services/baseQuery.js

import httpRequest from "@/utils/httpRequest";

const baseQuery = async (args) => {
    // Kiểm tra args có phải là một object hay là chuỗi endpoint
    const isObject = typeof args === "object";

    /* Cấu hình cho config của Axios với các field cần có */
    const config = {
        url: isObject ? args.url : args,
        method: isObject ? args.method : "GET",
    };

    /* Các field optional */
    if (isObject) {
        if (args.body) config.data = args.body;
        if (args.headers) config.headers = args.headers;

        // CẬP NHẬT: Config param
        if (args.params) config.params = args.params;
    }

    /* Thực hiện gọi API */
    try {
        const data = await httpRequest(config);
        return { data };
    } catch (error) {
        return { error: error.response?.data || error.message };
    }
};

export default baseQuery;



In [ ]:
// src/services/product.js

import { createApi } from "@reduxjs/toolkit/query/react";
import baseQuery from "./baseQuery";

// 1. Tạo endpoint
const END_POINT = "/products";

export const productApi = createApi({
    reducerPath: "productApi",
    baseQuery,
    // 2. Định nghĩa Tag để quản lý cache
    tagTypes: ["Products"],

    endpoints: (builder) => ({
        /* GET: Lấy danh sách products */
        getProducts: builder.query({
            // 4. Các tham số params: Nhận đối số khi gọi customHook
            query: (params) => ({
                url: END_POINT,
                method: "GET",
                params: params,
            }),
            // 3. Dán nhãn Tag để nhận biết dữ liệu này được Cache
            providesTags: ["Products"],
        }),

        /* POST: Tạo mới product */
        createProduct: builder.mutation({
            query: (body) => ({
                url: END_POINT,
                method: "POST",
                body,
            }),
            // 5. Báo hiệu để re-fetch lại danh sách products
            invalidatesTags: ["Products"],
        }),

        /* 6. PATCH: Cập nhật sản phẩm */
        updateProduct: builder.mutation({
            query: ({ id, ...body }) => ({
                url: `${END_POINT}/${id}`,
                method: "PATCH",
                body,
            }),
            invalidatesTags: ["Products"],
        }),

        /* 7. DELETE: Xoá */
        deleteProduct: builder.mutation({
            query: ({ id }) => ({
                url: `${END_POINT}/${id}`,
                method: "DELETE",
            }),
            invalidatesTags: ["Products"],
        }),
    }),
});

export const {
    useGetProductsQuery,
    useCreateProductMutation,
    useUpdateProductMutation,
    useDeleteProductMutation,
} = productApi;


In [ ]:
// src/pages/Home/index.jsx

import ProductModal from "@/components/ProductModal";
import { Button } from "@/components/ui/button";
import { cn } from "@/lib/utils";
import {
    useCreateProductMutation,
    useDeleteProductMutation,
    useGetProductsQuery,
    useUpdateProductMutation,
} from "@/services/product";
import React, { useState } from "react";
import { Link, useSearchParams } from "react-router";

function Home() {
    /* ==========================================================
     * State
     * ==========================================================*/

    /* Xác định page hiện tại */
    const [searchParams] = useSearchParams();
    const currentPage = Number(searchParams.get("page")) || 1;

    /* Lấy dữ liệu từ response API */
    const { data, isLoading, isFetching } = useGetProductsQuery({
        page: currentPage,
    });

    /* THÊM/SỬA/XOÁ */
    const [createProduct] = useCreateProductMutation();
    const [updateProduct] = useUpdateProductMutation();
    const [deleteProduct] = useDeleteProductMutation();

    /* State quản lý Modal */
    const [isModalOpen, setIsModalOpen] = useState(false);
    const [modalMode, setModalMode] = useState("create"); // create | edit
    const [selectedProduct, setSelectedProduct] = useState(null);

    /* ==========================================================
     * Logic
     * ==========================================================*/

    /* Xử lý MỞ Modal TẠO MỚI */
    const handleOpenCreate = () => {
        setModalMode("create");
        setIsModalOpen(true);
        setSelectedProduct(null);
    };

    /* Xử lý MỞ Modal CHỈNH SỬA */
    const handleOpenEdit = (product) => {
        setModalMode("edit");
        setSelectedProduct(product);
        setIsModalOpen(true);
    };

    /* Xử lý Submit Modal */
    const handleSubmitModal = async (formData) => {
        try {
            // Nếu ở chế độ TẠO MỚI
            if (modalMode === "create") {
                await createProduct(formData).unwrap();
                alert("Thêm sản phẩm thành công!");
            } else {
                // Nếu ở chế độ CHỈNH SỬA
                await updateProduct({
                    id: selectedProduct.id,
                    ...formData,
                }).unwrap();
                alert("Cập nhật sản phẩm thành công!");
            }

            // Đóng Modal
            setIsModalOpen(false);
        } catch (error) {
            console.error("Lỗi:", error);
            alert("Có lỗi xảy ra: " + JSON.stringify(error));
            return error;
        }
    };

    /* Xử lý khi XOÁ product */
    const handleDelete = async (id) => {
        if (window.confirm("Bạn có chắc muốn xoá sản phẩm này không?")) {
            try {
                await deleteProduct(id).unwrap();
                alert("Đã xoá thành công!");
            } catch (error) {
                alert("Xóa thất bại!");
                return error;
            }
        }
    };

    /* ==========================================================
     * JSX
     * ==========================================================*/

    /* Hiển thị Loading khi tải dữ liệu từ API */
    if (isLoading)
        return <div className="p-8 text-center">Đang tải dữ liệu...</div>;

    /* Lấy ra danh sách products */
    const products = data?.data?.items || [];

    /* Phân trang (Refactor theo mẫu pagination.jsx) */
    const renderPagination = () => {
        if (!data?.data?.pagination) return null;
        const { last_page } = data.data.pagination;

        return Array(last_page)
            .fill()
            .map((_, index) => {
                const pageNum = index + 1;
                const isActive = pageNum === currentPage;

                return (
                    <Link
                        key={index}
                        to={`?page=${pageNum}`}
                        className={cn(
                            "mx-1 inline-block min-w-9 rounded border border-gray-800 p-2 text-center text-sm font-medium transition-colors",
                            isActive
                                ? "border-orange-500 bg-orange-500 text-white"
                                : "bg-white text-gray-900 hover:bg-gray-100",
                        )}
                    >
                        {pageNum}
                    </Link>
                );
            });
    };

    return (
        <div className="container mx-auto p-4">
            {/* Heading: Tiêu đề + Nút Thêm Sản phẩm */}
            <div className="mb-6 flex items-center justify-between">
                <h1 className="text-2xl font-bold">Danh sách sản phẩm</h1>
                <Button onClick={handleOpenCreate}>+ Thêm sản phẩm</Button>
            </div>

            {/* Loading khi chuyển trang */}
            {isFetching && (
                <div className="mb-2 text-sm text-blue-500">
                    Đang cập nhật...
                </div>
            )}

            {/* Danh sách sản phẩm */}
            <div className="grid grid-cols-1 gap-6 sm:grid-cols-2 md:grid-cols-3 lg:grid-cols-4">
                {products.map((product) => (
                    <div
                        key={product.id}
                        className="flex flex-col rounded-lg border bg-white p-4 shadow-sm transition hover:shadow-md"
                    >
                        <div className="relative mb-4 aspect-square overflow-hidden rounded-md bg-gray-100">
                            <img
                                src={product.thumbnail}
                                alt={product.title}
                                className="h-full w-full object-cover"
                                onError={(e) => {
                                    e.target.src =
                                        "https://via.placeholder.com/200?text=No+Image";
                                }}
                            />
                        </div>

                        <h3 className="line-clamp-1 text-lg font-semibold">
                            {product.title}
                        </h3>
                        <p className="mb-2 text-sm text-gray-500">
                            {product.brand}
                        </p>
                        <p className="mb-4 font-bold text-blue-600">
                            ${product.price}
                        </p>

                        <div className="mt-auto flex gap-2">
                            <Button
                                variant="outline"
                                className="flex-1"
                                onClick={() => handleOpenEdit(product)}
                            >
                                Sửa
                            </Button>
                            <Button
                                variant="destructive"
                                className="flex-1"
                                onClick={() => handleDelete(product.id)}
                            >
                                Xóa
                            </Button>
                        </div>
                    </div>
                ))}
            </div>

            {/* Phân trang */}
            <div className="mt-8 flex flex-wrap justify-center gap-2">
                {renderPagination()}
            </div>

            {/* Modal */}
            <ProductModal
                open={isModalOpen}
                onClose={() => setIsModalOpen(false)}
                onSubmit={handleSubmitModal}
                initialData={selectedProduct}
                title={
                    modalMode === "create"
                        ? "Thêm sản phẩm mới"
                        : "Chỉnh sửa sản phẩm"
                }
            />
        </div>
    );
}

export default Home;


In [ ]:
//src/components/ProductModal/index.jsx

import { Description } from "@radix-ui/react-dialog";
import PropTypes from "prop-types";
import React, { useEffect, useState } from "react";
import {
    Dialog,
    DialogContent,
    DialogFooter,
    DialogHeader,
    DialogTitle,
} from "../ui/dialog";
import { Label } from "../ui/label";
import { Input } from "../ui/input";
import { Button } from "../ui/button";
import { Textarea } from "../ui/textarea";

function ProductModal({ open, onClose, onSubmit, initialData, title }) {
    /* ==========================================================
     * State
     * ==========================================================*/

    /* State lưu trữ dữ liệu từ formData */
    const [formData, setFormData] = useState({
        title: "",
        price: "",
        category: "",
        brand: "",
        description: "",
        thumbnail: "",
    });

    console.log(formData);

    /* State lỗi validate form */
    const [errors, setErrors] = useState({});

    /* Trạng thái ban đầu của form khi mở Modal */
    useEffect(() => {
        if (open) {
            // Nếu có sản phẩm được chọn
            if (initialData) {
                // Chế độ EDIT
                // Fill data cũ
                setFormData({
                    title: initialData.title || "",
                    price: initialData.price || 0,
                    category: initialData.category || "",
                    brand: initialData.brand || "",
                    description: initialData.description || "",
                    thumbnail: initialData.thumbnail || "",
                });
            } else {
                // Chế độ THÊM
                // Reset form
                setFormData({
                    title: "",
                    price: "",
                    category: "",
                    brand: "",
                    description: "",
                    thumbnail: "",
                });
            }

            // Xóa lỗi cũ
            setErrors({});
        }
    }, [open, initialData]);

    /* ==========================================================
     * Logic
     * ==========================================================*/

    /* Change Input */
    const handleChange = (e) => {
        const { name, value } = e.target;
        setFormData((prev) => ({ ...prev, [name]: value }));

        // Clear lỗi khi bắt đầu nhập
        if (errors[name]) {
            setErrors((prev) => ({ ...prev, [name]: null }));
        }
    };

    /* Xử lý Validate & Submit */
    const handleSubmit = () => {
        const newErrors = {};

        // Validate các trường bắt buộc
        if (!formData.title.trim())
            newErrors.title = "Tên sản phẩm là bắt buộc";
        if (!formData.price) newErrors.price = "Giá là bắt buộc";
        if (!formData.category.trim())
            newErrors.category = "Danh mục là bắt buộc";
        if (!formData.brand.trim()) newErrors.brand = "Thương hiệu là bắt buộc";
        if (!formData.description.trim())
            newErrors.description = "Mô tả là bắt buộc";

        // Nếu vẫn có lỗi thì không cho submit
        if (Object.keys(newErrors).length > 0) {
            setErrors(newErrors);
            return;
        }

        // Chuẩn bị payload để gửi đi
        const payload = {
            ...formData,
            price: Number(formData.price),
            thumbnail:
                formData.thumbnail.trim() ||
                `https://picsum.photos/200?random=${Date.now()}`,
            // Các trường required từ API => ẩn đi để không làm khó người dùng
            stock: 100,
            sku: `SKU-${Math.random().toString(36).substr(2, 9).toUpperCase()}`,
            brand: formData.brand || "No Brand",
            weight: 1,
            width: 10,
            height: 10,
            length: 10,
            minimumOrderQuantity: 1,
            returnPolicy: "No return policy",
            tags: ["general", "new"],
            discountPercentage: 5,
            rating: 5,
        };

        // Submit
        onSubmit(payload);
    };

    /* ==========================================================
     * JSX
     * ==========================================================*/
    return (
        <Dialog open={open} onOpenChange={onClose}>
            <DialogContent className="sm:max-w-[500px]">
                <DialogHeader>
                    <DialogTitle>{title}</DialogTitle>
                </DialogHeader>

                <div className="grid gap-4 py-4">
                    {/* Tên sản phẩm */}
                    <div className="grid gap-2">
                        <Label htmlFor="title" className="text-left">
                            Tên sản phẩm <span className="text-red-500">*</span>
                        </Label>
                        <Input
                            id="title"
                            name="title"
                            value={formData.title}
                            onChange={handleChange}
                            className={errors.title ? "border-red-500" : ""}
                        />
                        {errors.title && (
                            <span className="text-xs text-red-500">
                                {errors.title}
                            </span>
                        )}
                    </div>

                    {/* Giá & Thương hiệu (2 cột) */}
                    <div className="grid grid-cols-2 gap-4">
                        <div className="grid gap-2">
                            <Label htmlFor="price">
                                Giá ($) <span className="text-red-500">*</span>
                            </Label>
                            <Input
                                id="price"
                                name="price"
                                type="number"
                                value={formData.price}
                                onChange={handleChange}
                                className={errors.price ? "border-red-500" : ""}
                            />
                            {errors.price && (
                                <span className="text-xs text-red-500">
                                    {errors.price}
                                </span>
                            )}
                        </div>
                        <div className="grid gap-2">
                            <Label htmlFor="brand">
                                Thương hiệu{" "}
                                <span className="text-red-500">*</span>
                            </Label>
                            <Input
                                id="brand"
                                name="brand"
                                value={formData.brand}
                                onChange={handleChange}
                                className={errors.brand ? "border-red-500" : ""}
                            />
                            {errors.brand && (
                                <span className="text-xs text-red-500">
                                    {errors.brand}
                                </span>
                            )}
                        </div>
                    </div>

                    {/* Danh mục */}
                    <div className="grid gap-2">
                        <Label htmlFor="category">
                            Danh mục <span className="text-red-500">*</span>
                        </Label>
                        <Input
                            id="category"
                            name="category"
                            value={formData.category}
                            onChange={handleChange}
                            className={errors.category ? "border-red-500" : ""}
                        />
                        {errors.category && (
                            <span className="text-xs text-red-500">
                                {errors.category}
                            </span>
                        )}
                    </div>

                    {/* Ảnh URL */}
                    <div className="grid gap-2">
                        <Label htmlFor="thumbnail">
                            Hình ảnh URL (Tùy chọn)
                        </Label>
                        <Input
                            id="thumbnail"
                            name="thumbnail"
                            placeholder="Để trống sẽ tự sinh ảnh ngẫu nhiên"
                            value={formData.thumbnail}
                            onChange={handleChange}
                        />
                    </div>

                    {/* Mô tả */}
                    <div className="grid gap-2">
                        <Label htmlFor="description">
                            Mô tả <span className="text-red-500">*</span>
                        </Label>
                        <Textarea
                            id="description"
                            name="description"
                            value={formData.description}
                            onChange={handleChange}
                            className={
                                errors.description ? "border-red-500" : ""
                            }
                        />
                        {errors.description && (
                            <span className="text-xs text-red-500">
                                {errors.description}
                            </span>
                        )}
                    </div>
                </div>

                <DialogFooter>
                    <Button variant="outline" onClick={onClose}>
                        Hủy
                    </Button>
                    <Button onClick={handleSubmit}>Lưu</Button>
                </DialogFooter>
            </DialogContent>
        </Dialog>
    );
}

/* Prop-types */
ProductModal.propTypes = {
    open: PropTypes.bool.isRequired,
    onClose: PropTypes.func.isRequired,
    onSubmit: PropTypes.func.isRequired,
    initialData: PropTypes.object,
    title: PropTypes.string,
};

export default ProductModal;
